<a href="https://colab.research.google.com/github/MaxGhislain/NLP_Models/blob/main/nlpProject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from IPython.display import display,HTML
from fastbook import *
from fastai.text.all import *

In [ ]:
path = Path('gdrive/My Drive/ColabDataFolders/NLP/sentimentanalysistweets')

In [ ]:
#Parse pos and neg tweets
positive = (path/'pos_tweets.txt').read_text().split('\n')
#positive[:5]

negative = (path/'neg_tweets.txt').read_text().split('\n')
#negative[:5]

In [ ]:
#Convert to dataframes
pos_df = pd.DataFrame(positive, columns =['tweet'])
neg_df = pd.DataFrame(negative, columns =['tweet'])

In [ ]:
#Label dataframes
pos_df['Class'] = 1
neg_df['Class'] = 0

#Combine dataframes
df = pd.concat([pos_df, neg_df])
df

,tweet,Class
0,""" I cheer myself up when I'm down by listening to my playlist called, Genius: Ballads and Cellos. I love my iPod and my taste of music.""",1
1,""" just watched the movie Wanted... it was pretty darn good.""",1
2,""" now I'm happy """,1
3,"""--plotting like i'm mike..'game plan:pass the ball to lebron AT ALL TIMES and DONT FOUL'..certainly we'll win haha..go cavs goooo!""",1
4,"""@ mcdonalds with my litto sis aka cuzin lol cristyyyyy """,1
...,...,...
1382,"""You can call me a HATER but..... The CAVS SUCK! """,0
1383,"""Youtube is being as sporadic as twitter right about now. """,0
1384,"”@RubyRose1 awww wish i could go! but its in sydney """,0
1385,"”laying in bed with no voice.. """,0


In [ ]:
#Create datablock
dls_clas = DataBlock(blocks=(TextBlock.from_df('tweet'), CategoryBlock),
    get_x=ColReader('text'),
    get_y=ColReader('Class'),
    splitter=RandomSplitter(0.2)
).dataloaders(df, bs=128, seq_len=80)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos "" xxunk xxmaj hah , yea , i suppose "" xxunk haha i xxup wish i xxunk meet you .. you should stop by seattle some time home of the xxup xxunk "" xxunk bed time . i wish i did n't have an exam on thursday xxunk i would just go around xxunk people to xxunk ! lol . xxmaj night guys ! """,1
1,"xxbos "" ok … i had 2 come xxunk early from xxunk coz of hayfever xxmaj anyway i was back at xxunk "" ; "" i wanted the cavs to win it all ! xxmaj xxunk xxunk the game xxmaj cavs … . i do n't like xxunk and i know this is alot of xxunk on xxmaj bron . """,0
2,"xxbos "" xxunk xxmaj hey , hey , hey , hey ! xxmaj i 'm ur xxunk xxunk . @oprah sent me . "" xxunk for me please , the ex is xxunk to start xxunk * * at my / our babies 1st xxmaj birthday party . what a xxunk . and i still have a headache """,1


In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()


In [ ]:
#Refine Model
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.650760,0.609881,0.717500,00:04


In [ ]:
#learn.unfreeze()
#learn.fit_one_cycle(5, 2e-3)

In [ ]:
#Gradual Unfreezing w/ slice function

learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.548231,0.512886,0.802500,00:04


In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.492097,0.485312,0.762500,00:05


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.416996,0.448022,0.800000,00:06
1,0.400869,0.434228,0.810000,00:06


In [ ]:
learn.save('../gdrive/My Drive/ColabDataFolders/NLP/sentimentanalysistweets/models/1epoch32-2')

Path('models/../gdrive/My Drive/ColabDataFolders/NLP/sentimentanalysistweets/models/1epoch32-2.pth')

In [ ]:
learn = learn.load('../gdrive/My Drive/ColabDataFolders/NLP/sentimentanalysistweets/models/1epoch32-2')

In [ ]:
#Test Neural Network
tweet = "I hate children"

pred_class,pred_idx,probabilities = learn.predict(tweet)
pred_class

'0'

In [ ]:
#Fast Setup
from IPython.display import display,HTML
from fastbook import *
from fastai.text.all import *

path = Path('gdrive/My Drive/ColabDataFolders/NLP/sentimentanalysistweets')

#Parse pos and neg tweets
positive = (path/'pos_tweets.txt').read_text().split('\n')
#positive[:5]

negative = (path/'neg_tweets.txt').read_text().split('\n')
#negative[:5]

#Convert to dataframes
pos_df = pd.DataFrame(positive, columns =['tweet'])
neg_df = pd.DataFrame(negative, columns =['tweet'])

#Label dataframes
pos_df['Class'] = 1
neg_df['Class'] = 0

#Combine dataframes
df = pd.concat([pos_df, neg_df])
#df

#Create datablock
dls_clas = DataBlock(blocks=(TextBlock.from_df('tweet'), CategoryBlock),
    get_x=ColReader('text'),
    get_y=ColReader('Class'),
    splitter=RandomSplitter(0.2)
).dataloaders(df, bs=128, seq_len=80)

learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

learn = learn.load('../gdrive/My Drive/ColabDataFolders/NLP/sentimentanalysistweets/models/1epoch32-2')

#Test Neural Network
tweet = "This movie made me very sad and makes me want to throw up"

pred_class,pred_idx,probabilities = learn.predict(tweet)
pred_class

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


'0'

In [ ]:
#Mock UI
"""
strList = ["I agree with this",
           "I love him",
           "I hate him",
           "He should die"
           ]
negList = []

for comment in strList:
    pred_class,pred_idx,probabilities = learn.predict(comment)
    if pred_class == "0":
      negList.append(comment)

print("Negative comments: ")
for comment in negList:
    print(comment)
    print("Do you agree with the assessment? 1 for yes 0 for no")
    x = input()
    if x == "0":
      negList.remove(comment)

print("Final neg list: ")
for i in negList:
    print(i)
"""
#Mock UI
strList = ["I agree with this",
           "I love him",
           "I hate him",
           "He should die"
           ]
negList = []
finalList = []

for comment in strList:
    pred_class,pred_idx,probabilities = learn.predict(comment)
    if pred_class == "0":
      negList.append(comment)

print("Negative comments: " + str(len(negList)))
for comment in negList:
    print(comment)
    print("Do you agree with the assessment? 1 for yes 0 for no")
    if input() == "1":
      finalList.append(comment)

print("Final list: ")
for i in finalList:
    print(i)

Negative comments: 2
I hate him
Do you agree with the assessment? 1 for yes 0 for no
1
He should die
Do you agree with the assessment? 1 for yes 0 for no
1
Final list: 
I hate him
He should die


In [ ]:
from fastai.vision.widgets import *

learn_inf = learn.load('../gdrive/My Drive/ColabDataFolders/NLP/sentimentanalysistweets/models/1epoch32-2')

btn_upload = widgets.FileUpload()

btn_run = widgets.Button(description='Classify')

out_pl = widgets.Output()
lbl_pred = widgets.Label()

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

VBox([widgets.Label('Select your bear!'),
      btn_upload, btn_run, out_pl, lbl_pred])

In [ ]:
#Test UI
from fastai.vision.widgets import *

text = widgets.Textarea(description='Enter text',value="1\n2\n3")

btn_run = widgets.Button(description='Classify')

out_pl = widgets.Output()
lbl_pred = widgets.Label()

def on_click_classify(change):
  t = text.value
  out_pl.clear_output()
  for line in t.splitlines():
      pred,pred_idx,probs = learn.predict(line)
      if pred_class == "0":
        negList.append(line)
      #lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
      with out_pl: print(line + "\n" + f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}')

btn_run.on_click(on_click_classify)

VBox([widgets.Label('Sentiment check program'),
      text, btn_run, out_pl])

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


...

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


In [ ]:
#Test UI w/ Mock UI
from fastai.vision.widgets import *

text = widgets.Textarea(description='Enter text',value="1\n2\n3")

btn_run = widgets.Button(description='Classify')

out_pl = widgets.Output()
lbl_pred = widgets.Label()

negList = []
finalList = []

def on_click_classify(change):
  t = text.value
  for line in t.splitlines():
      pred,pred_idx,probs = learn.predict(line)
      if pred_class == "0":
        negList.append(line)
      #lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
      with out_pl: print(line + "\n" + f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}' + "\nDo you agree with the assessment? 1 for yes 0 for no")

btn_run.on_click(on_click_classify)

VBox([widgets.Label('Sentiment check program'),
      text, btn_run, out_pl])

#print("Negative comments: " + str(len(negList)))

for comment in negList:
  text = widgets.Text
    with out_pl: print(comment + "\nDo you agree with the assessment? 1 for yes 0 for no")
    if input() == "1":
      finalList.append(comment)

for i in finalList:
    with out_pl: print(i)

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


EOFError: ignored

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


EOFError: ignored